In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import tensorflow as tf
from tensorflow.python.data import Dataset
import keras
from keras.utils import to_categorical
from keras import models
from keras import layers

C:\Users\amanda\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Load and preprocess data 

In [3]:
df_train = pd.read_csv('train.csv', index_col=0)
df_test = pd.read_csv('test.csv', index_col=0)

In [4]:
# identify and replace missing values
print(df_train.isnull().sum())
print(df_test.isnull().sum())

print(df_train['opened_position_qty '].mean())
print(df_train['opened_position_qty '].median())

print(df_train['closed_position_qty'].mean())
print(df_train['closed_position_qty'].median())

# replace missing values with median (less sensitive to outliers)
df_train['opened_position_qty '].fillna(df_train['opened_position_qty '].median(),inplace=True)
df_test['opened_position_qty '].fillna(df_train['opened_position_qty '].median(),inplace=True)
df_train['closed_position_qty'].fillna(df_train['closed_position_qty'].median(),inplace=True)
df_test['closed_position_qty'].fillna(df_train['closed_position_qty'].median(), inplace=True)

last_price                   0
mid                          0
opened_position_qty     120982
closed_position_qty     120982
transacted_qty               0
d_open_interest              0
bid1                         0
bid2                         0
bid3                         0
bid4                         0
bid5                         0
ask1                         0
ask2                         0
ask3                         0
ask4                         0
ask5                         0
bid1vol                      0
bid2vol                      0
bid3vol                      0
bid4vol                      0
bid5vol                      0
ask1vol                      0
ask2vol                      0
ask3vol                      0
ask4vol                      0
ask5vol                      0
y                            0
dtype: int64
last_price                  0
mid                         0
opened_position_qty     51478
closed_position_qty     51478
transacted_qty              0


In [5]:
# Normalize data
from sklearn import preprocessing

x_train = df_train[df_train.columns[:26]]
y_train = df_train['y']
x_test = df_test

# Normalize training data by subtracting mean and scaling to unit variance
std_scale = preprocessing.StandardScaler().fit(x_train)
x_train_norm = std_scale.transform(x_train)
x_train = pd.DataFrame(x_train_norm, index=x_train.index, columns=x_train.columns)

# Normalize testing data by using mean and SD of training set
x_test_norm = std_scale.transform(x_test)
x_test = pd.DataFrame(x_test_norm, index=x_test.index, columns=x_test.columns) 

In [6]:
model = keras.Sequential([
 keras.layers.Dense(64, activation=tf.nn.relu, input_shape=(x_train.shape[1],)),
 keras.layers.Dense(128, activation=tf.nn.relu),
 keras.layers.Dense(256, activation=tf.nn.relu),
 keras.layers.Dense(512, activation=tf.nn.relu),
 keras.layers.Dense(2, activation=  'softmax')
 ])

In [7]:
model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
train_model = model.fit(
 x_train, y_train,
 epochs= 30, batch_size = 128)

Epoch 1/30
401223/401223 [==============================] - 25s 63us/step - loss: 0.6245 - acc: 0.6524
Epoch 2/30
401223/401223 [==============================] - 24s 60us/step - loss: 0.6215 - acc: 0.6548
Epoch 3/30
401223/401223 [==============================] - 24s 61us/step - loss: 0.6209 - acc: 0.6555
Epoch 4/30
401223/401223 [==============================] - 23s 57us/step - loss: 0.6204 - acc: 0.6559
Epoch 5/30
401223/401223 [==============================] - 25s 63us/step - loss: 0.6202 - acc: 0.6562
Epoch 6/30
401223/401223 [==============================] - 20s 50us/step - loss: 0.6199 - acc: 0.6564
Epoch 7/30
401223/401223 [==============================] - 24s 59us/step - loss: 0.6196 - acc: 0.6565
Epoch 8/30
401223/401223 [==============================] - 22s 55us/step - loss: 0.6194 - acc: 0.6568
Epoch 9/30
401223/401223 [==============================] - 19s 47us/step - loss: 0.6192 - acc: 0.6572
Epoch 10/30
401223/401223 [==============================] - 20s 51us/ste

## Save results

In [9]:
df_test['Predicted'] = model.predict_proba(x_test)[:,1]
df_test[['Predicted']].to_csv('submission.csv')